<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ─── SETUP FOR DASHBOARD ───

import os, sys

try:
    # Step 1: Clone the GitHub repository if not already present
    if not os.path.exists("/content/Cloud_Computing"):
        !git clone https://github.com/Method-for-Software-System-Development/Cloud_Computing.git /content/Cloud_Computing

    # Step 2: Change directory to project root
    %cd /content/Cloud_Computing

    # Step 3: Checkout the 'develop' branch
    !git fetch origin -q
    !git checkout develop -q

    # Step 4: Add 'logic' directory to Python path
    sys.path.append("/content/Cloud_Computing/logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio

    import gradio as gr

    # Step 6: Import required notebooks from 'logic'
    from importnb import Notebook
    with Notebook():
        import user_controller as uc
        import mqqt_sim as mqtt_module

    print("✅ Setup completed successfully.")

except Exception as e:
    print("❌ Setup failed:", str(e))

Cloning into '/content/Cloud_Computing'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 169 (delta 24), reused 19 (delta 19), pack-reused 142 (from 1)
Receiving objects: 100% (169/169), 4.64 MiB | 12.73 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/Cloud_Computing
lior_levi 200
noa_katz 200
matan_tal 200


In [3]:
!pip install -U gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00


In [18]:
import gradio as gr

# State variable to track login status
is_logged_in = gr.State(False)

# Main Gradio UI
with gr.Blocks(css=r"""
:root {
  --brand-blue:   #0B4F8F;
  --panel-blue:   #B4CDEA;
}

#header {
  background: var(--brand-blue);
  padding: 0.8rem 1.2rem;
  display: flex;
  align-items: center;
  min-height: 100px;
  color: white;
}

#right_header_panel {
  display: flex;
  flex-direction: column;
  align-items: flex-end;
}
""") as demo:

    # --------------- HEADER ----------------
    with gr.Row(elem_id="header", equal_height=True):
        with gr.Column(scale=3):
            gr.HTML("""
            <div>
              <h1 style="margin:0;font-size:2rem;font-weight:800;">OptiLine</h1>
              <h3 style="margin:0;font-size:1.2rem;opacity:0.8;">CIM & Robotics Lab - Braude College of Engineering</h3>
            </div>
            """)
        with gr.Column(scale=1, elem_id="right_header_panel"):
            quit_btn = gr.Button("Quit")

    # --------------- MAIN DISPLAY ZONES ----------------

    with gr.Row():

        # ─── LEFT PANEL: before login ───
        with gr.Column(scale=1, visible=True) as left_login_panel:
            gr.Markdown("### Sign In:")
            username   = gr.Textbox(label="Username")
            password   = gr.Textbox(label="Password", type="password")
            login_msg  = gr.Markdown("")
            with gr.Row():
                clear_btn  = gr.Button("Clear")
                submit_btn = gr.Button("Submit")
            gr.Markdown("""
            **OptiLine** is your real-time production dashboard …
            """)

        # ─── LEFT PANEL: after login ───
        with gr.Column(scale=1, visible=False) as left_dashboard_panel:
            welcome_txt = gr.Markdown("")
            gr.Markdown("#### Menu:\n> Search Engine\n> Statistics\n> Simulate Problems")
            leaderboard_txt = gr.Markdown("")

        # ─── RIGHT PANEL: before login ───
        with gr.Column(scale=2, visible=True) as right_info_panel:
            gr.Markdown("""
            ### About the system
            - High-frequency sensor data …
            """)

        # ─── RIGHT PANEL: after login ───
        with gr.Column(scale=2, visible=False) as right_sensor_panel:
            sensor_txt = gr.Markdown("")        # ← live data goes here
            gr.Markdown("* Temperature: 24.7 °C\n* Humidity: 53%\n* Pressure: 1001 hPa")


    # --------------- LOGIC ----------------

    def clear_fields():
        return "", ""

    def do_login(user, pw):
      """
      Handles user login logic and updates the dashboard panels accordingly.
      Retrieves the user's profile and generates a dynamic leaderboard table with medals.

      Args:
          user (str): The username provided in the login form.
          pw   (str): The password provided in the login form.

      Returns:
          list: Gradio UI updates for all relevant components (panels and texts).
      """
      if uc.login(user, pw):
          # Retrieve the user's profile from Firebase
          _, profile = uc.fb.get_user(user)
          name = profile.get("first_name", user)
          role = profile.get("role", "")
          user_name = profile.get("username", user)

          # Create a personalized welcome message
          welcome_text = f"### Welcome back, **{name}** ({role})"

          # Get the dynamic leaderboard from Firebase, including the current user's rank
          leaderboard_list, user_rank = uc.get_leaderboard(user_name)

          # Build a Markdown table for the leaderboard, with medals for top 3
          leaderboard_md = "#### Leaderboard\n\n| Rank | Name | Score |\n|:----:|:-----------|:------:|\n"
          for rank, uname, score in leaderboard_list:
              # Assign a medal emoji to the top 3
              medal = ""
              if rank == 1: medal = "🥇"
              elif rank == 2: medal = "🥈"
              elif rank == 3: medal = "🥉"

              # Highlight the current user in bold with "(You)" label
              you = " <b>(You)</b>" if uname == user_name else ""
              name_str = f"**{uname}{you}**" if uname == user_name else uname
              score_str = f"**{score}**" if uname == user_name else str(score)

              leaderboard_md += f"| {rank} {medal} | {name_str} | {score_str} |\n"

          # Example sensor display (can be replaced with real data)
          sensor_text = "* Temperature: 24.7 °C\n* Humidity: 53%\n* Pressure: 1001 hPa"

          # Return Gradio UI updates:
          # - Hide the login panel
          # - Show the dashboard panel
          # - Hide the info panel
          # - Show the sensor panel
          # - Set is_logged_in to True
          # - Clear login message
          # - Update welcome message, leaderboard, and sensor info
          return [
              gr.update(visible=False),  # hide login form
              gr.update(visible=True),   # show dashboard
              gr.update(visible=False),  # hide info panel
              gr.update(visible=True),   # show sensor panel
              True,                      # is_logged_in
              "",                        # clear login_msg
              gr.update(value=welcome_text),         # welcome_txt
              gr.update(value=leaderboard_md),       # leaderboard_txt
              gr.update(value=sensor_text)           # sensor_txt
          ]
      else:
          # If login fails, show an error message and do not update other panels
          return [
              gr.update(), gr.update(), gr.update(), gr.update(),
              False, "Incorrect username or password",
              gr.update(), gr.update(), gr.update()
          ]


    # Button callbacks
    clear_btn.click(fn=clear_fields, outputs=[username, password])

    submit_btn.click(
        fn=do_login,
        inputs=[username, password],
        outputs=[
            left_login_panel,
            left_dashboard_panel,
            right_info_panel,
            right_sensor_panel,
            is_logged_in,
            login_msg,
            welcome_txt,
            leaderboard_txt,    # << כאן
            sensor_txt
        ]
    )


    quit_btn.click(
        fn=do_logout,
        outputs=[
            left_login_panel,
            left_dashboard_panel,
            right_info_panel,
            right_sensor_panel,
            is_logged_in,
            login_msg
        ]
    )

# Launch app
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32af7168dbedaf4e5b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
